In [1]:
import pandas 

import twon_agents

In [2]:
SEED: int = 1
DATA_PATH: str = "../../../data"

In [3]:
congress_meta: pandas.DataFrame = (
    pandas.read_csv(f"{DATA_PATH}/raw/twitter.english/congress_meta.csv")
    .pipe(lambda _df: _df.assign(
        username=_df["link"].str.extract(r"https://x.com/(\w+)"),
        first_name=_df["name"].str.extract(r"\w+, (.+)"),
        last_name=_df["name"].str.extract(r"(.+), \w+"),
        party=_df["party"].map(dict(D="Democratic Party", R="Republican Party"))
    ))
    [["username", "first_name", "last_name", "party"]]
)
congress_meta.head()

,username,first_name,last_name,party
0,SenatorBaldwin,Tammy,Baldwin,Democratic Party
1,SenJohnBarrasso,John,Barrasso,Republican Party
2,SenatorBennet,Michael F.,Bennet,Democratic Party
3,MarshaBlackburn,Marsha,Blackburn,Republican Party
4,SenBlumenthal,Richard,Blumenthal,Democratic Party


In [4]:
posts: pandas.DataFrame = twon_agents.data.filter_tweets(
    pandas.read_csv(f"{DATA_PATH}/raw/twitter.english/USCongressTweets_2023.csv")
    .rename(columns={"tweet_id": "id", "full_text": "text", "screen_name": "username", "original_user_id": "author_id"})
    .merge(
        right=congress_meta,
        left_on="username",
        right_on="username"
    )
    [["id", "author_id",  "username", "first_name", "last_name", "party", "text", "created_at"]]
    .pipe(lambda _df: _df.assign(id=_df["id"].astype('Int64'), author_id=_df["author_id"].astype('Int64')))
)
posts

,id,author_id,username,first_name,last_name,party,text,created_at
12,1692273284526555392,21406834,SenMarkey,Edward,Markey,Democratic Party,Mifepristone for medication abortion is still ...,2023-08-17 20:32:46+00:00
15,1692224923949805824,21406834,SenMarkey,Edward,Markey,Democratic Party,YouTube and Google can't keep treating young p...,2023-08-17 17:20:36+00:00
17,1691854007113842688,21406834,SenMarkey,Edward,Markey,Democratic Party,Thank you to the movement that got us here. An...,2023-08-16 16:46:43+00:00
18,1691854006153347072,21406834,SenMarkey,Edward,Markey,Democratic Party,Even as we face record-breaking heat and extre...,2023-08-16 16:46:42+00:00
19,1691854005176074496,21406834,SenMarkey,Edward,Markey,Democratic Party,"And every day since, the IRA has turned words ...",2023-08-16 16:46:42+00:00
...,...,...,...,...,...,...,...,...
91766,1506739661519081472,983348251972816768,SenHydeSmith,Cindy,Hyde-Smith,Republican Party,Instead of acknowledging its failing energy po...,2022-03-23 21:08:23+00:00
91767,1506722331510591488,983348251972816768,SenHydeSmith,Cindy,Hyde-Smith,Republican Party,"Whether it’s farming, research, agricultural e...",2022-03-23 19:59:32+00:00
91768,1506682090124947712,983348251972816768,SenHydeSmith,Cindy,Hyde-Smith,Republican Party,We need a strong response to the heavy-handed ...,2022-03-23 17:19:37+00:00
91773,1505981160790315008,983348251972816768,SenHydeSmith,Cindy,Hyde-Smith,Republican Party,New York’s lack of religious exemption in its ...,2022-03-21 18:54:23+00:00


In [10]:
(
    posts
    [["id", "author_id", "first_name", "last_name", "username", "party", "created_at", "text"]]
    .to_csv(f"{DATA_PATH}/interim/twitter.english.posts.csv")
)